In [60]:
# Manipulação e visualização de dados
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Bibliotecas para aprendizado de máquina
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

# MLflow para rastreamento de experimentos
import mlflow
print(mlflow.__version__)

# Supressão de avisos
import warnings
warnings.filterwarnings("ignore")

3.5.1


In [61]:
# Carregar o dataset
dados = pd.read_csv(
    "mymoviedb.csv",
    sep=",",            # ou ";" se for ponto e vírgula
    engine="python",    # parser alternativo
    encoding="utf-8"    # ou "latin1" se der erro
)

print("First 5 records:", dados.head())

# Exibir informações sobre o dataset
print('Dados info:', dados.info())

# Exibir dimensões do dataset
print('Dimensões:', dados.shape)

First 5 records:   Release_Date                    Title  \
0   2021-12-15  Spider-Man: No Way Home   
1   2022-03-01               The Batman   
2   2022-02-25                  No Exit   
3   2021-11-24                  Encanto   
4   2021-12-22           The King's Man   

                                            Overview  Popularity Vote_Count  \
0  Peter Parker is unmasked and no longer able to...    5083.954       8940   
1  In his second year of fighting crime, Batman u...    3827.658       1151   
2  Stranded at a rest stop in the mountains durin...    2618.087        122   
3  The tale of an extraordinary family, the Madri...    2402.201       5076   
4  As a collection of history's worst tyrants and...    1895.511       1793   

  Vote_Average Original_Language                               Genre  \
0          8.3                en  Action, Adventure, Science Fiction   
1          8.1                en            Crime, Mystery, Thriller   
2          6.3                en 

In [62]:
# Exibir estatísticas básicas
display(dados.describe())

# Verificar valores ausentes
display(dados.isnull().sum().sort_values(ascending=False))

,Popularity
count,9827.000000
mean,40.320570
std,108.874308
min,7.100000
25%,16.127500
50%,21.191000
75%,35.174500
max,5083.954000


Genre                11
Poster_Url           11
Popularity           10
Vote_Count           10
Vote_Average         10
Original_Language    10
Title                 9
Overview              9
Release_Date          0
dtype: int64

In [63]:
dados_limpos = dados.copy()

# Remover duplicatas
antes = dados_limpos.shape[0]
dados_limpos.drop_duplicates(inplace=True)
depois = dados_limpos.shape[0]
print(f'Duplicatas removidas: {antes - depois}')

# Imputação para colunas numéricas com mediana
num_cols = dados_limpos.select_dtypes(include=[np.number]).columns
medianas = dados_limpos[num_cols].median()
dados_limpos[num_cols] = dados_limpos[num_cols].fillna(medianas)

# Imputação para colunas categóricas com moda (valor mais frequente)
#cat_cols = dados_limpos.select_dtypes(include=['object', 'category', 'bool']).columns
#for c in cat_cols:
    #moda = dados_limpos[c].mode(dropna=True)
    #if not moda.empty:
    #    dados_limpos[c] = dados_limpos[c].fillna(moda.iloc[0])
    #else:
#        dados_limpos[c] = dados_limpos[c].fillna('Desconhecido')

dados_limpos = dados_limpos.dropna(subset=['Title'])

# Preencher colunas categóricas (object) com 'Desconhecido'
cols_categoricas = dados_limpos.select_dtypes(include='object').columns

dados_limpos[cols_categoricas] = dados_limpos[cols_categoricas].fillna('Desconhecido')

print('Valores ausentes restantes (depois da imputação):')
display(dados_limpos.isnull().sum().sort_values(ascending=False).head(10))

# Converte para numérico
dados_limpos['Vote_Count'] = pd.to_numeric(dados_limpos['Vote_Count'], errors='coerce')
dados_limpos['Vote_Average'] = pd.to_numeric(dados_limpos['Vote_Average'], errors='coerce')

# Encontrar a linha com a maior votação
filme_vote_count = dados_limpos.loc[dados_limpos['Vote_Count'].idxmax()]

print("Filme com a maior votação:")
print(filme_vote_count[['Title', 'Vote_Count']])

# Encontrar a linha com a maior nota
filme_top = dados_limpos.loc[dados_limpos['Vote_Average'].idxmax()]

print("Filme com a maior nota:")
print(filme_top[['Title', 'Vote_Average']])

Duplicatas removidas: 0
Valores ausentes restantes (depois da imputação):


Release_Date         0
Title                0
Overview             0
Popularity           0
Vote_Count           0
Vote_Average         0
Original_Language    0
Genre                0
Poster_Url           0
dtype: int64

Filme com a maior votação:
Title         Inception
Vote_Count      31077.0
Name: 380, dtype: object
Filme com a maior nota:
Title           Kung Fu Master Huo Yuanjia
Vote_Average                          10.0
Name: 9401, dtype: object


In [64]:
#Transforme as variaveis categóricas de linguagem e genero em númericas (utilize dummy)
dados_onehot = dados.copy()

# ONE-HOT ENCODING: cria colunas dummies para categorias
dados_onehot = pd.get_dummies(dados_onehot, columns=['Original_Language', 'Genre'], drop_first=False, dtype=int)
print('Formato após One-Hot:', dados_onehot.shape)
display(dados_onehot.head())

Formato após One-Hot: (9837, 2388)


,Release_Date,Title,Overview,Popularity,Vote_Count,Vote_Average,Poster_Url,Original_Language_ar,Original_Language_bn,Original_Language_ca,...,"Genre_Western, Drama","Genre_Western, Drama, Action, Adventure","Genre_Western, Drama, Adventure","Genre_Western, Drama, Crime","Genre_Western, Drama, History","Genre_Western, Drama, Mystery","Genre_Western, History","Genre_Western, Horror","Genre_Western, Mystery, Thriller, Drama","Genre_Western, Thriller"
0,2021-12-15,Spider-Man: No Way Home,Peter Parker is unmasked and no longer able to...,5083.954,8940,8.3,https://image.tmdb.org/t/p/original/1g0dhYtq4i...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2022-03-01,The Batman,"In his second year of fighting crime, Batman u...",3827.658,1151,8.1,https://image.tmdb.org/t/p/original/74xTEgt7R3...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2022-02-25,No Exit,Stranded at a rest stop in the mountains durin...,2618.087,122,6.3,https://image.tmdb.org/t/p/original/vDHsLnOWKl...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021-11-24,Encanto,"The tale of an extraordinary family, the Madri...",2402.201,5076,7.7,https://image.tmdb.org/t/p/original/4j0PNHkMr5...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-12-22,The King's Man,As a collection of history's worst tyrants and...,1895.511,1793,7.0,https://image.tmdb.org/t/p/original/aq4Pwv5Xeu...,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
#Normalize as variaveis numéricas
dados_scaled_minmax = dados_onehot.copy()

num_cols = dados_onehot.select_dtypes(include=[np.number]).columns

minmax = MinMaxScaler()
dados_scaled_minmax[num_cols] = minmax.fit_transform(dados_scaled_minmax[num_cols])

print('Visualização após Normalização (Min-Max):')
display(dados_scaled_minmax[num_cols].describe().T.head())

Visualização após Normalização (Min-Max):


,count,mean,std,min,25%,50%,75%,max
Popularity,9827.0,0.006544,0.021445,0.0,0.001778,0.002776,0.00553,1.0
Original_Language_ar,9837.0,0.000203,0.014258,0.0,0.000000,0.000000,0.00000,1.0
Original_Language_bn,9837.0,0.000102,0.010083,0.0,0.000000,0.000000,0.00000,1.0
Original_Language_ca,9837.0,0.000102,0.010083,0.0,0.000000,0.000000,0.00000,1.0
Original_Language_cn,9837.0,0.011081,0.104685,0.0,0.000000,0.000000,0.00000,1.0


In [66]:
# Criar um único dataset final e salvar em 'dados_processados.csv'
# Estratégia:
# - Base: `dados_limpos`
# - Categóricas: One-Hot Encoding (sem dropar categoria)

if 'dados_limpos' not in globals():
    raise RuntimeError("dados_limpos não está disponível. Execute as etapas de limpeza antes desta célula.")

# Identificar colunas de tipos originais (antes do one-hot)
cat_cols_orig = dados_limpos.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

# One-Hot nas categóricas
_df = pd.get_dummies(dados_limpos, columns=cat_cols_orig, drop_first=False, dtype=int)

# Renomear dataset final
dados_processados = _df.copy()

# Salvar
processed_data_path = "dados_processados.csv"
dados_processados.to_csv(processed_data_path, index=False)
print(f"dados_processados salvo: {processed_data_path} | shape: {dados_processados.shape}")

dados_processados salvo: dados_processados.csv | shape: (9828, 37444)


In [67]:
# Salvar o dataset processado como um arquivo CSV
processed_data_path = "dados_processados.csv"
dados.to_csv(processed_data_path, index=False)
print("Dataset processado salvo localmente.")

# Registrar o dataset processado como artefato no MLflow
mlflow.start_run()  # Iniciar um novo experimento
mlflow.log_artifact(processed_data_path)  # Registrar o arquivo como artefato
mlflow.end_run()  # Encerrar o experimento

print("Features armazenadas e versionadas com sucesso no MLflow!")

Dataset processado salvo localmente.
Features armazenadas e versionadas com sucesso no MLflow!
